In [5]:
import datasets

In [6]:
from huggingface_hub import notebook_login
notebook_login()

In [7]:
from datasets import load_dataset, concatenate_datasets


data_pun = load_dataset('ai4bharat/samanantar', 'pa')
data_hin = load_dataset('ai4bharat/samanantar', 'hi')
data_mar = load_dataset('ai4bharat/samanantar', 'mr')

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1491: FutureWarning: The repository for ai4bharat/samanantar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ai4bharat/samanantar
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1491: FutureWarning: The repository for ai4bharat/samanantar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ai4bharat/samanantar
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [17]:
# Select 100,000 samples from each dataset
data_pun_sample = data_pun['train'].shuffle(seed=42).select(range(100000))
data_hin_sample = data_hin['train'].shuffle(seed=42).select(range(100000))
data_mar_sample = data_mar['train'].shuffle(seed=42).select(range(100000))

def add_prefix(example, prefix):
    example['instruction'] = prefix 
    return example

data_pun_sample = data_pun_sample.map(lambda example: add_prefix(example, 'Translate to Punjabi->'))
data_hin_sample = data_hin_sample.map(lambda example: add_prefix(example, 'Translate to Hindi->'))
data_mar_sample = data_mar_sample.map(lambda example: add_prefix(example, 'Translate to Marathi->'))

data_full = concatenate_datasets([data_pun_sample, data_hin_sample, data_mar_sample])



Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [18]:
data_full = data_full.shuffle(seed=42)
data_full = data_full.rename_column('src', 'inputs')
data_full = data_full.rename_column('tgt', 'outputs')
data_full = data_full.remove_columns(['idx'])
print(data_full[:10])

{'inputs': ['The expression could simply mean family members but could also include slaves or employees.', 'Ive never seen it.', "I haven't had any contact with them.", 'The municipality contains 10 districts.', 'Hence a solution should be found immediately.', "But not all attention will be on Pant as there's a rival to him for the wicketkeeper batsman's spot in Jharkhand's Ishan Kishan, his teammate in the World Cup U-19 side and also an attacking batsman like the Delhi player.", 'You dont have heart?', 'But Hemant Karkare was a selfless and sincere police officer who laid his life for the security of the nation.', 'In the first century C.E., Christs followers needed to put up a vigorous fight to keep true worship free of corrupt teachings and attitudes.', 'In Mumbai, a sick newborn child died after a hospital refused admission as the parents didnt have legal tender.'], 'outputs': ['ठीक - ठीक किन लोगों से स्तिफनास का ‘ कुटुम्ब ’ बना था, यह बताया नहीं गया है ।', 'मी वापरुन पाहिलेला नाह

In [19]:
def create_prompt(example):
    bos_token = "<s>"
    eos_token = "</s>"
    text = []
    instructions = example['instruction']
    inputs = example['inputs']
    outputs = example['outputs']
    
    for ins, inp, out in zip(instructions, inputs, outputs):
        full_prompt = ""
        full_prompt += bos_token
        full_prompt += "[INST]"
        full_prompt += ins +'\n'
        full_prompt += inp
        full_prompt += "[/INST]" 
        full_prompt += "\n" + out
        full_prompt += eos_token
        text.append(full_prompt)
    
    example['text'] = text

    return example

data_full = data_full.map(create_prompt, batched=True)
data_full = data_full.remove_columns(['instruction'])
data_full = data_full.remove_columns(['inputs'])
data_full = data_full.remove_columns(['outputs'])

Map:   0%|          | 0/300000 [00:00<?, ? examples/s]

In [20]:
print(data_full['text'][0])

<s>[INST]Translate to Hindi->
The expression could simply mean family members but could also include slaves or employees.[/INST]
ठीक - ठीक किन लोगों से स्तिफनास का ‘ कुटुम्ब ’ बना था, यह बताया नहीं गया है ।</s>


In [21]:
data_full.push_to_hub('abh1shank/project_lang_data',private=True,commit_message="data_for_multilingual")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/300 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/279 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/abh1shank/project_lang_data/commit/a4b0bbab07702159421131882fbd63652386785f', commit_message='data_for_multilingual', commit_description='', oid='a4b0bbab07702159421131882fbd63652386785f', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
dataset = load_dataset('abh1shank/project_lang_data')

Generating train split:   0%|          | 0/300000 [00:00<?, ? examples/s]

In [23]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 300000
    })
})

In [25]:
dataset['train']['text'][:20]

['<s>[INST]Translate to Hindi->\nThe expression could simply mean family members but could also include slaves or employees.[/INST]\nठीक - ठीक किन लोगों से स्तिफनास का ‘ कुटुम्ब ’ बना था, यह बताया नहीं गया है ।</s>',
 '<s>[INST]Translate to Marathi->\nIve never seen it.[/INST]\nमी वापरुन पाहिलेला नाही.</s>',
 "<s>[INST]Translate to Marathi->\nI haven't had any contact with them.[/INST]\nमाझा आणि त्यांचा काही संपर्क झालेला नाही.</s>",
 '<s>[INST]Translate to Punjabi->\nThe municipality contains 10 districts.[/INST]\nਇਹ ਡਵੀਜ਼ਨ ਹੈ ਜਿਸ ਵਿੱਚ 10 ਜ਼ਿਲ੍ਹੇ ਹਨ।</s>',
 '<s>[INST]Translate to Marathi->\nHence a solution should be found immediately.[/INST]\nत्यामुळे यावर त्वरीत उपाययोजना करण्याची गरज आहे.</s>',
 "<s>[INST]Translate to Punjabi->\nBut not all attention will be on Pant as there's a rival to him for the wicketkeeper batsman's spot in Jharkhand's Ishan Kishan, his teammate in the World Cup U-19 side and also an attacking batsman like the Delhi player.[/INST]\nਟੀਮ 'ਚ ਵਿਕਟਕੀਪਰ ਬੱਲੇਬਾਜ਼ ਦੀ 